# CARLA Autonomous Vehicle with LiDAR-Camera Fusion

**AI-controlled vehicle using LiDAR-to-camera projection for obstacle detection and navigation**

Output: `output/YYYY-MM-DD_HH-MM-SS/lidar_projection/camera_{front,left,right}/`

## 1. Imports & Server Connection

In [1]:
import time
import numpy as np
import logging

from src import (
    Config, connect_to_carla, setup_synchronous_mode, restore_async_mode,
    clear_environment, reload_environment, spawn_vehicle, spawn_cameras,
    spawn_lidar, create_sensor_queues, project_lidar_to_camera,
    save_projection_image, update_spectator_view,
    lidar_density_to_waypoint, TrajectoryController,
)
from src.sensors import create_camera_callback, create_lidar_callback
from src.projection import image_to_array

# Configure logging for debugging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(name)s] %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('carla_navigation.log')
    ]
)
logger = logging.getLogger('Navigation')

config = Config()

# Connect to CARLA and enable synchronous mode
client, world = connect_to_carla(config)
original_settings, traffic_manager = setup_synchronous_mode(world, client, config)

logger.info(f"Connected to CARLA | Map: {world.get_map().name} | {config.fps} FPS sync mode")
print(f"✓ Connected to CARLA | Map: {world.get_map().name} | {config.fps} FPS sync mode")

2026-01-27 17:36:24,996 [Navigation] INFO: Connected to CARLA | Map: Carla/Maps/Town10HD_Opt | 20 FPS sync mode


✓ Connected to CARLA | Map: Carla/Maps/Town10HD_Opt | 20 FPS sync mode


---

## 2. Main Pipeline

Setup environment, spawn vehicle & sensors, run simulation with LiDAR-based steering.

In [ ]:
# Environment setup (optional: clear/reload)
clear_environment(world)  # Remove buildings, props, etc.
# reload_environment(world)  # Uncomment to restore

# Spawn vehicle and sensors
vehicle = spawn_vehicle(world, config)
cameras, K = spawn_cameras(world, vehicle, config)
lidar = spawn_lidar(world, vehicle, config)
update_spectator_view(world, vehicle)
world.tick()

# Setup sensor queues and output directory
camera_queues, lidar_queue = create_sensor_queues(cameras)
output_dir = config.get_output_dir()
output_dirs = config.create_output_dirs(output_dir)

for cam_name, cam in cameras.items():
    cam.listen(create_camera_callback(camera_queues, cam_name))
lidar.listen(create_lidar_callback(lidar_queue))

print(f"✓ Vehicle + {len(cameras)} cameras + LiDAR spawned")
print(f"✓ Output: {output_dir}/")

✓ Vehicle + 3 cameras + LiDAR spawned
✓ Output: output/2026-01-27_17-39-53/


: 

In [ ]:
# Initialize trajectory controller
trajectory_controller = TrajectoryController(config)

# Run simulation with waypoint-based navigation
frame_counter, tick_count, last_print_time = 0, 0, 0
current_projection_counts = {}
current_waypoint = None
current_control = None

logger.info(f"Starting simulation | Duration: {config.simulation_duration}s | Throttle: {config.throttle}")
print(f"✓ Running {config.simulation_duration}s | Throttle: {config.throttle} | Steering strategy: Waypoint-based\n")

try:
    while tick_count < config.total_ticks:
        world.tick()
        tick_count += 1
        
        # Wait for sensor data
        start = time.time()
        while time.time() - start < 2.0:
            if all(camera_queues[c].qsize() > 0 for c in cameras) and lidar_queue.qsize() > 0:
                break
            time.sleep(0.001)
        
        should_save = (tick_count % config.capture_interval_ticks == 0)
        
        # Collect and clear queues
        camera_images = {n: camera_queues[n].get() for n in cameras if not camera_queues[n].empty()}
        for q in camera_queues.values():
            while not q.empty(): q.get()
        lidar_data = lidar_queue.get() if not lidar_queue.empty() else None
        while not lidar_queue.empty(): lidar_queue.get()
        
        # Process sensors
        if len(camera_images) == len(cameras) and lidar_data:
            projection_counts = {}
            for cam_name in cameras:
                proj_array, _, weighted_density = project_lidar_to_camera(
                    lidar_data, lidar, cameras[cam_name], image_to_array(camera_images[cam_name]), K, config)
                projection_counts[cam_name] = weighted_density
                if should_save:
                    save_projection_image(cam_name, proj_array, frame_counter, output_dirs)
            
            if should_save:
                frame_counter += 1
            current_projection_counts = projection_counts
        
        # === NEW: Waypoint-based navigation pipeline ===
        if current_projection_counts:
            # Step 1: Convert LiDAR density to target waypoint
            waypoint_local, direction, analysis = lidar_density_to_waypoint(
                current_projection_counts, vehicle, config
            )
            current_waypoint = waypoint_local
            
            # Step 2: Convert waypoint to vehicle control
            control, control_data = trajectory_controller.waypoint_to_control(
                waypoint_local, vehicle, analysis
            )
            current_control = control_data
            
            # Step 3: Apply control to vehicle
            vehicle.apply_control(control)
            
            logger.debug(
                f"Tick {tick_count} | Waypoint: ({waypoint_local[0]:.2f}m, {waypoint_local[1]:.2f}m) | "
                f"{trajectory_controller.get_control_summary(control_data)}"
            )
        
        update_spectator_view(world, vehicle)
        
        # Status every 5s
        sim_time = tick_count * config.fixed_delta_seconds
        if int(sim_time) % 5 == 0 and int(sim_time) != last_print_time:
            last_print_time = int(sim_time)
            vel = vehicle.get_velocity()
            speed = 3.6 * np.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
            l, f, r = [current_projection_counts.get(k, 0) for k in ['left', 'front', 'right']]
            
            waypoint_str = "N/A"
            if current_waypoint is not None:
                waypoint_str = f"({current_waypoint[0]:.1f}m, {current_waypoint[1]:.2f}m)"
            
            control_str = "N/A"
            if current_control is not None:
                control_str = trajectory_controller.get_control_summary(current_control)
            
            logger.info(
                f"[{int(sim_time)}s] Speed: {speed:.1f} km/h | Densities L:{l:.0f} F:{f:.0f} R:{r:.0f} | "
                f"Waypoint: {waypoint_str} | {control_str}"
            )
            print(f"[{int(sim_time)}s] {speed:.1f} km/h | L:{l:.0f} F:{f:.0f} R:{r:.0f} | Waypoint: {waypoint_str} | {control_str}")

except KeyboardInterrupt:
    logger.warning("Simulation interrupted by user")
    print("\n✗ Interrupted")

logger.info(f"Simulation complete | {frame_counter} frames saved")
print(f"\n✓ Done! {frame_counter} frames saved")

2026-01-27 17:39:53,225 [Navigation] INFO: Starting simulation | Duration: 30.0s | Throttle: 0.3


✓ Running 30.0s | Throttle: 0.3 | Steering strategy: Waypoint-based



2026-01-27 17:39:53,436 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19072.0 F:15266.0 R:18967.0] | Lateral offset: 0.000m
2026-01-27 17:39:53,436 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 2.45m/s
2026-01-27 17:39:53,437 [Navigation] DEBUG: Tick 1 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:2.5m/s
2026-01-27 17:39:53,591 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19055.0 F:15089.0 R:19318.0] | Lateral offset: 0.000m
2026-01-27 17:39:53,592 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 2.94m/s
2026-01-27 17:39:53,592 [Navigation] DEBUG: Tick 2 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:2.9m/s
2026-01-27 17:39:53,744 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densitie

[5s] 17.3 km/h | L:19354 F:15277 R:19112 | Waypoint: (10.0m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:4.8m/s


2026-01-27 17:40:09,851 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:18891.0 F:15525.0 R:19025.0] | Lateral offset: 0.000m
2026-01-27 17:40:09,852 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 4.85m/s
2026-01-27 17:40:09,852 [Navigation] DEBUG: Tick 102 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:4.9m/s
2026-01-27 17:40:10,003 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19026.0 F:15508.0 R:19111.0] | Lateral offset: 0.000m
2026-01-27 17:40:10,004 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 4.87m/s
2026-01-27 17:40:10,004 [Navigation] DEBUG: Tick 103 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:4.9m/s
2026-01-27 17:40:10,160 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Dens

...

2026-01-27 17:40:42,265 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19055.0 F:15612.0 R:19121.0] | Lateral offset: 0.000m
2026-01-27 17:40:42,266 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 5.12m/s
2026-01-27 17:40:42,266 [Navigation] DEBUG: Tick 302 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.1m/s
2026-01-27 17:40:42,426 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19062.0 F:15352.0 R:19443.0] | Lateral offset: 0.000m
2026-01-27 17:40:42,427 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 5.00m/s
2026-01-27 17:40:42,427 [Navigation] DEBUG: Tick 303 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.0m/s
2026-01-27 17:40:42,590 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Dens

[20s] 18.4 km/h | L:18999 F:15553 R:19231 | Waypoint: (10.0m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.1m/s


2026-01-27 17:40:57,828 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19289.0 F:15298.0 R:19117.0] | Lateral offset: 0.000m
2026-01-27 17:40:57,829 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 5.11m/s
2026-01-27 17:40:57,830 [Navigation] DEBUG: Tick 402 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.1m/s
2026-01-27 17:40:58,013 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:18959.0 F:15005.0 R:19330.0] | Lateral offset: 0.000m
2026-01-27 17:40:58,013 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 5.10m/s
2026-01-27 17:40:58,014 [Navigation] DEBUG: Tick 403 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.1m/s
2026-01-27 17:40:58,159 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Dens

[25s] 18.3 km/h | L:18955 F:16383 R:19262 | Waypoint: (10.0m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.1m/s


2026-01-27 17:41:13,019 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19006.0 F:16433.0 R:19188.0] | Lateral offset: 0.000m
2026-01-27 17:41:13,020 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 5.02m/s
2026-01-27 17:41:13,020 [Navigation] DEBUG: Tick 502 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.0m/s
2026-01-27 17:41:13,197 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Densities [L:19279.0 F:16577.0 R:19297.0] | Lateral offset: 0.000m
2026-01-27 17:41:13,197 [TrajectoryControl] DEBUG: Control conversion | Cross-track: 0.000m | Steering: 0.000 | Throttle: 0.300 | Brake: 0.000 | Speed: 5.08m/s
2026-01-27 17:41:13,198 [Navigation] DEBUG: Tick 503 | Waypoint: (10.00m, 0.00m) | Steer:+0.00 Throttle:0.30 Brake:0.00 Speed:5.1m/s
2026-01-27 17:41:13,341 [WaypointPlanning] DEBUG: Waypoint decision | Direction: STRAIGHT | Dens

---

## 3. Cleanup & Shutdown

Stop sensors, destroy actors, restore async mode.

In [4]:
restore_async_mode(world, original_settings, traffic_manager)

for cam in cameras.values():
    cam.stop()
lidar.stop()
time.sleep(0.5)

for cam in cameras.values():
    cam.destroy()
lidar.destroy()
vehicle.destroy()
world.tick()

print(f"✓ Cleanup done | Output: {output_dir}/lidar_projection/")

✓ Cleanup done | Output: output/2026-01-27_13-47-46/lidar_projection/
